In [ ]:
from tensorflow.python.client import device_lib
for d in device_lib.list_local_devices():
    print(d.name, d.physical_device_desc)

In [ ]:
from keras import backend as K
cfg = K.tf.ConfigProto()
cfg.gpu_options.allow_growth = True
K.set_session(K.tf.Session(config=cfg))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras import optimizers
import keras.backend as K
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.image import ImageDataGenerator
from sklearn.datasets import load_iris
from keras.datasets import fashion_mnist, mnist, cifar100

np.set_printoptions(suppress=True)
%matplotlib inline

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [ ]:
train_generator = train_datagen.flow_from_directory(
        'kaggle/dogs-vs-cats/train',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

In [ ]:
input_shape = (150, 150, 3)

In [ ]:
model = Sequential()

model.add(Conv2D(128, (3,3), activation='relu', input_shape=input_shape, name='L0'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(256, (3,3), activation='relu', name='L1'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(512, (3,3), activation='relu', name='L2'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=2))

model.add(Flatten())         
model.add(Dense(1024, activation='relu')) 
model.add(Dropout(0.2))                  
model.add(Dense(1, activation='sigmoid'))

opt = optimizers.RMSprop(lr=0.001)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000,
        epochs=5)